In [1]:
!pip3 install --quiet twint

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.4/385.4 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 5.3 MB/s eta 0:00:00


In [ ]:
import twint

In [ ]:
!pip3 install --quiet snscrape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import datetime
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='whitegrid')

In [ ]:
text = input('Enter query text to be matched (or leave it blank by pressing enter)') 

username = input('Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): ') 

since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

count = int(input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 

retweet = input('Exclude Retweets? (y/n): ') 

replies = input('Exclude Replies? (y/n): ') 

Enter query text to be matched (or leave it blank by pressing enter)cybercrime
Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): 
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): 2022-01-01
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): 2022-01-10
Enter max number of tweets or enter -1 to retrieve all possible tweets: 1000
Exclude Retweets? (y/n): y
Exclude Replies? (y/n): y


In [ ]:
def search(text, username, since, until, retweet, replies):
    global filename
    q = text
    if username != '':
        q += f" from:{username}"
    if until == '':
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
    q += f" until:{until}"
    if since == '':
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - datetime.timedelta(days=7), '%Y-%m-%d')
    q += f" since:{since}"
    if retweet == 'y':
        q += f" exclude:retweets"
    if replies == 'y':
        q += f" exclude:replies"
    if username != '' and text != '':
        filename = f"{since}_{until}_{username}_{text}.csv"
    elif username != "":
        filename = f"{since}_{until}_{username}.csv"
    else:
        filename = f"{since}_{until}_{text}.csv"
    print(filename)
    return q

In [ ]:
q = search(text, username, since, until, retweet, replies)

# Creating list to append tweet data
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
if count == -1:
    for i, tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([
            tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang,
            tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
            tweet.quoteCount, tweet.media
        ])
else:
    with tqdm_notebook(total=count) as pbar:
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): 
            # declare a username
            if i >= count: # number of tweets you want to scrape
                break
            tweets_list1.append([
                tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang,
                tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
                tweet.quoteCount, tweet.media
            ])
            pbar.update(1)

# Creating a dataframe from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username', 'Language',
                                                 'Hashtags', 'ReplyCount', 'RetweetCount',
                                                 'LikeCount', 'QuoteCount', 'Media'])

2022-01-01_2022-01-10_cybercrime.csv


  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
tw_df = tweets_df1.sort_values(by='DateTime',ascending=False) 
tw_df.tail()

,DateTime,TweetId,Text,Username,Language,Hashtags,ReplyCount,RetweetCount,LikeCount,QuoteCount,Media
995,2022-01-07 06:19:26+00:00,1479336860082073600,"लड़की कर रही थी लड़की को डेट, जब खुलासा हुआ तो...",aajtak,hi,"[cybercrime, onlinedating]",4,3,57,1,None
996,2022-01-07 06:14:54+00:00,1479335719696879617,FortiGuard Labs Prediksi Serangan Siber pada T...,TechforID,in,"[FortiGuardLabs, SeranganSiber, CyberCrime, Cy...",1,3,1,0,None
997,2022-01-07 06:14:38+00:00,1479335651732430850,"Conducted awareness on Cyber Crime, Traffic Ru...",pskkp_cyb,en,None,2,7,67,0,[Photo(previewUrl='https://pbs.twimg.com/media...
998,2022-01-07 06:12:01+00:00,1479334991146504193,Some simple and useful tips on identifying a P...,celminestrategy,en,"[Phishing, CyberCrime, OnlineSafety, Informati...",0,2,2,0,[Photo(previewUrl='https://pbs.twimg.com/media...
999,2022-01-07 06:09:30+00:00,1479334360507576321,Cyber Crime &amp; Digital Evidence [Indian Per...,busicat_adv,en,None,0,0,0,0,[Photo(previewUrl='https://pbs.twimg.com/media...


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# keyword pencarian tweet
keyword = "data breach"

# tanggal awal dan akhir pencarian tweet
since_date = "2022-03-01"
until_date = "2022-03-02"

# list untuk menyimpan data tweet
tweets_list = []

# melakukan scraping tweet
for tweet in sntwitter.TwitterSearchScraper(keyword + f" since:{since_date} until:{until_date}").get_items():
    # menyimpan data tweet yang diperlukan
    tweet_dict = {
        'date': tweet.date.strftime('%Y-%m-%d'),
        'id': tweet.id,
        'content': tweet.rawContent,
        'user': tweet.user.username,
        'url': tweet.url,
        'reply_count': tweet.replyCount,
        'retweet_count': tweet.retweetCount,
        'like_count': tweet.likeCount
    }
    tweets_list.append(tweet_dict)

# mengubah list tweet menjadi dataframe
tweets_df = pd.DataFrame(tweets_list)

# menampilkan 10 tweet teratas
print(tweets_df.head(10))


In [ ]:
tweets_df.tail()

In [ ]:
import nltk
nltk.download()

In [ ]:
import snscrape.modules.twitter as sntwitter

max_tweets = 50
search_words = "data breach"

tweets = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_words + ' lang:en since:2020-01-01 until:2021-01-01').get_items()):
    if i>max_tweets:
        break
    tweets.append({'date': tweet.date, 'content': tweet.rawContent})

tweets = pd.DataFrame(tweets)

# menampilkan 10 tweet teratas
tweets.head(10)

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load dataset
df = tweets

# Remove URLs, mentions, and hashtags
df['cleaned_text'] = df['content'].apply(lambda x: re.sub(r"http\S+", "", x))
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: re.sub(r"@\S+", "", x))
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: re.sub(r"#\S+", "", x))

# Convert text to lowercase
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: x.lower())

# Tokenize text
df['tokenized_text'] = df['cleaned_text'].apply(lambda x: word_tokenize(x))

# Remove stop words
stop_words = set(stopwords.words('english'))
df['filtered_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word not in stop_words])

# Lemmatize text
lemmatizer = WordNetLemmatizer()
df['lemmatized_text'] = df['filtered_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Save preprocessed data
df.to_csv('preprocessed_tweet_data.csv', index=False)


In [ ]:
dfs = pd.read_csv('preprocessed_tweet_data.csv')
dfs.tail()

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('preprocessed_tweet_data.csv')

# Buat kolom baru untuk menyimpan hasil anotasi
df['annotated_data'] = ''

# Lakukan anotasi
for index, row in df.iterrows():
    print(row['cleaned_text'])
    label = input("Label: ")
    df.at[index, 'label'] = label
    
# Simpan hasil anotasi ke dalam file CSV
df.to_csv('annotated_tweet_data.csv', index=False)


In [ ]:
dfa = pd.read_csv('annotated_tweet_data.csv')
dfa.head()

In [ ]:
dfa = dfa.rename(columns={'content': 'tweet'})
dfa.head()


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# Load stopwords
stop_words = stopwords.words('english')


# Tokenize text and remove stop words
corpus = dfa['tweet'].tolist()
tokens = [word_tokenize(text.lower()) for text in corpus]
filtered_tokens = [[word for word in token if word not in stop_words] for token in tokens]
filtered_corpus = [' '.join(token) for token in filtered_tokens]

# Extract features using Bag of Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(filtered_corpus)
y = dfa['label'].tolist()

print(X.toarray())
print(y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score


# Split dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Calculate evaluation metrics
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

# Print evaluation metrics
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

# Print classification report
#print(classification_report(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB

# Split dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model using Multinomial Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict using testing set
y_pred = clf.predict(X_test)

# Evaluate model using confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# Evaluate model using k-fold cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=kfold)
print("Cross validation scores: ", scores)
print("Average cross validation score: ", np.mean(scores))


In [ ]:
from sklearn.metrics import classification_report

# Load test dataset
test_df = pd.read_csv('annotated_tweet_data.csv')

# Tokenize text and remove stop words
test_corpus = test_df['content'].tolist()
test_tokens = [word_tokenize(text.lower()) for text in test_corpus]
test_filtered_tokens = [[word for word in token if word not in stop_words] for token in test_tokens]
test_filtered_corpus = [' '.join(token) for token in test_filtered_tokens]

# Extract features using Bag of Words
test_X = vectorizer.transform(test_filtered_corpus)
test_y = test_df['label'].tolist()

# Predict on test dataset
test_y_pred = clf.predict(test_X)

# Print classification report
print(classification_report(test_y, test_y_pred, zero_division=0))


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load stopwords
stop_words = stopwords.words('english')

# Load dataset from csv file
df = pd.read_csv('annotated_tweet_data.csv')

# Tokenize text and remove stop words
corpus = df['content'].tolist()
tokens = [word_tokenize(text.lower()) for text in corpus]
filtered_tokens = [[word for word in token if word not in stop_words] for token in tokens]
filtered_corpus = [' '.join(token) for token in filtered_tokens]

# Extract features using Bag of Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(filtered_corpus)
y = df['label'].tolist()

# Train model
clf = MultinomialNB()
clf.fit(X, y)

# Make predictions
new_text = ['The users as a whole, not the individual. So I can be harmed if someone else falls for this malicious attack. It is similar to when Google has a databreach, compromising several thousands of accounts at once.']
new_tokens = [word_tokenize(text.lower()) for text in new_text]
new_filtered_tokens = [[word for word in token if word not in stop_words] for token in new_tokens]
new_filtered_corpus = [' '.join(token) for token in new_filtered_tokens]
new_X = vectorizer.transform(new_filtered_corpus)

prediction = clf.predict(new_X)

print(prediction)


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load stopwords
stop_words = stopwords.words('english')

# Load dataset from csv file
df = pd.read_csv('annotated_tweet_data.csv')

# Tokenize text and remove stop words
corpus = df['content'].tolist()
tokens = [word_tokenize(text.lower()) for text in corpus]
filtered_tokens = [[word for word in token if word not in stop_words] for token in tokens]
filtered_corpus = [' '.join(token) for token in filtered_tokens]

# Extract features using Bag of Words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(filtered_corpus)
y = df['label'].tolist()

# Train model
clf = MultinomialNB()
clf.fit(X, y)

# Make predictions
new_text = ['The users as a whole, not the individual. So I can be harmed if someone else falls for this malicious attack. It is similar to when Google has a databreach, compromising several thousands of accounts at once.'
            , 'I received a check of 17 dollars from the Equifax data breach settlement and it bounced because they used SIGNATURE BANK.'
            , 'I am monitoring the data breach hotline this week ama'
            , 'LockBit Ransomware Claims Data Breach at SpaceX Contractor. Hackers News'
            , 'Students in the Los Angeles school district faced a data breach last year, but now, hackers are taking mental health problems to a new level by selling off the data to buyers on the #darkweb.'
            , '@DPCIreland⁩ Can you please get someone in your office to respond to my e-mails in relation to a possible Data Breach'
            ]
new_tokens = [word_tokenize(text.lower()) for text in new_text]
new_filtered_tokens = [[word for word in token if word not in stop_words] for token in new_tokens]
new_filtered_corpus = [' '.join(token) for token in new_filtered_tokens]
new_X = vectorizer.transform(new_filtered_corpus)

# Predict and print results
prediction = clf.predict(new_X)

# Create pandas dataframe to store the results
results_df = pd.DataFrame({'Text': new_text, 'Label': prediction})

# Add column for label name
label_names = {0: 'Non-Attack', 1: 'Attack', 2: 'Neutral'}
results_df['Category'] = results_df['Label'].map(label_names)

results_df


###Test2


In [2]:
!pip install --quiet snscrape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 6.8 MB/s eta 0:00:00


In [3]:
!pip install -U --quiet scikit-learn

In [4]:
!pip install --quiet scikit-learn pandas numpy

In [5]:
import pandas as pd
import re
import numpy as np
import snscrape.modules.twitter as sntwitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Tentukan keyword dan tanggal pencarian
keyword = "pelayanan bank bri"
since_date = "2022-01-01"
until_date = "2023-01-01"

# Buat query pencarian
query = f"{keyword} since:{since_date} until:{until_date} lang:id"

# Scraping tweet dengan snscrape
tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    tweets.append([tweet.date, tweet.rawContent, tweet.user.username, tweet.likeCount, tweet.replyCount, tweet.retweetCount])
    if i >= 10000:
        break

# Simpan hasil scraping ke dalam dataframe
df = pd.DataFrame(tweets, columns=["Tanggal", "Tweet", "Username", "Likes", "Replies", "Retweets"])


In [7]:
# Menyimpan dataset dengan kolom sentimen baru
df.to_csv("data.csv", index=False)

In [9]:
df.tail()

,Tanggal,Tweet,Username,Likes,Replies,Retweets
1158,2022-01-10 04:26:17+00:00,"Bca emang terbaik dalam pelayanan, tapi dari s...",Thebensky,0,0,0
1159,2022-01-04 03:59:03+00:00,Ke @BANKBRI_ID cab Cilandak KKO di handle deng...,nyanyu_id,0,3,0
1160,2022-01-03 06:26:10+00:00,Personil Satuan Samapta Polres Nunukan melaksa...,PolresNunukan,0,0,0
1161,2022-01-03 03:33:51+00:00,@BANKBRI_ID @kontakBRI Mohon untuk ditingkatka...,Darto001,0,7,0
1162,2022-01-03 00:49:13+00:00,@BANKBRI_ID Semoga beruntung aamiin \nPelayana...,AkuuXrc,0,0,0


In [10]:
# Fungsi untuk melakukan preprocessing
def preprocess_tweet(tweet):
    # Menghapus tanda baca dan karakter khusus
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Mengubah ke huruf kecil
    tweet = tweet.lower()

    # Menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    tokens = nltk.word_tokenize(tweet)
    tokens = [token for token in tokens if token not in stop_words]
    
    # Melakukan lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Menggabungkan kembali token menjadi kalimat
    tweet = ' '.join(tokens)
    return tweet

# Preprocessing pada kolom Tweet dalam DataFrame
df['Tweet'] = df['Tweet'].apply(preprocess_tweet)


In [11]:
from textblob import TextBlob

# Fungsi untuk mendapatkan sentimen dari tweet
def get_sentiment(tweet):
    sentiment = TextBlob(tweet).sentiment.polarity
    if sentiment > 0:
        return 'positif'
    elif sentiment < 0:
        return 'negatif'
    else:
        return 'netral'

# Menentukan label sentimen untuk setiap tweet
df['Sentiment'] = df['Tweet'].apply(get_sentiment)
df.head()

,Tanggal,Tweet,Username,Likes,Replies,Retweets,Sentiment
0,2022-12-31 18:51:16+00:00,kontakbri bankbri_id promo_bri bank bumn pelay...,RaffaDwiputtra,0,1,0,positif
1,2022-12-29 04:47:45+00:00,bankbri_id spallpilllll min saran gak kalo lho...,KiKYyy,1,4,0,netral
2,2022-12-27 07:43:58+00:00,gandeng bank bri rutan 1 medan maksimalkan pel...,rutan1medan,0,0,0,netral
3,2022-12-26 10:38:50+00:00,poetra1804 informasikan jam operasional bank b...,BANKBRI_ID,1,1,0,netral
4,2022-12-26 07:07:18+00:00,kontakbri siangsiang darah c bri unit buludoan...,AsbudiAswan,3,13,0,netral


In [12]:
df.to_csv("sentimented_data.csv", index=False)

In [ ]:
# Membuat vektor fitur menggunakan CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Tweet'])

# Menentukan target
y = df['Sentiment']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Melakukan prediksi pada data uji
y_pred = clf.predict(X_test)

# Menghitung akurasi
acc = accuracy_score(y_test, y_pred)
print("Akurasi:", acc)


Akurasi: 0.922077922077922


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

# Membuat model SVM
svm_model = SVC(kernel='linear', probability=True)

# Membuat model Naive Bayes
nb_model = MultinomialNB()

# Menggabungkan kedua model dalam voting classifier
voting_model = VotingClassifier(estimators=[('svm', svm_model), ('nb', nb_model)], voting='soft')

# Melatih model pada data training
voting_model.fit(X_train, y_train)


VotingClassifier(estimators=[('svm', SVC(kernel='linear', probability=True)),
                             ('nb', MultinomialNB())],
                 voting='soft')

In [ ]:
from sklearn.metrics import accuracy_score

# Melakukan prediksi sentimen pada data testing
y_pred = voting_model.predict(X_test)

# Menghitung akurasi prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi prediksi: {accuracy}")


Akurasi prediksi: 0.958041958041958


In [ ]:
# Prediksi sentimen pada data baru
new_tweet = "Di RSX tmpatku dari 8 pasien covid19, yg 6 mninggal sbb diracuni. Stlah minum obat kansgung mulut berbusa. Lalu gk lama kmudian tahu2 dibawain keranda mayat. Yg dua lagi brontak.  Gn mo nelen obat. Dn slama sampai kini..."
new_tweet = preprocess_tweet(new_tweet)
X_test = vectorizer.transform([new_tweet])
y_pred = voting_model.predict(X_test)

# Cetak hasil prediksi
print("Hasil prediksi sentimen:")
print("Sentimen: ", y_pred[0])


Hasil prediksi sentimen:
Sentimen:  netral


In [ ]:
def preprocess_tweet(tweet):
    # Menghapus karakter non-angka/non-huruf
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    # Mengubah semua huruf menjadi lowercase
    tweet = tweet.lower()
    return tweet

# Menerapkan preprocessing pada tweet
df['Tweet'] = df['Tweet'].apply(preprocess_tweet)

# Membagi dataset menjadi data training dan data testing
X_train, X_test, y_train, y_test = train_test_split(df['Tweet'], df['Sentimen'], test_size=0.2, random_state=42)

# Melatih model Naive Bayes Classifier
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Melakukan transform pada data testing
X_test = vectorizer.transform(X_test)

# Melakukan prediksi sentimen
y_pred = nb.predict(X_test)

# Menghitung akurasi model
accuracy = np.mean(y_pred == y_test)

# Menampilkan hasil akurasi
print("Akurasi model:", accuracy)

KeyError: ignored

In [ ]:
# Membaca dataset
df_test = pd.read_csv("sentimen_hasil.csv")

# Preprocessing tweet
def preprocess_tweet(tweet):
    # Menghapus karakter non-angka/non-huruf
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    # Mengubah semua huruf menjadi lowercase
    tweet = tweet.lower()
    return tweet

# Menerapkan preprocessing pada tweet
df_test['Tweet'] = df_test['Tweet'].apply(preprocess_tweet)

# Menghitung rata-rata likes, replies, dan retweets
mean_likes = df_test['Likes'].mean()
mean_replies = df_test['Replies'].mean()
mean_retweets = df_test['Retweets'].mean()

# Menandai tweet yang memiliki like, reply, atau retweet di atas rata-rata sebagai sentimen positif
df_test['Sentimen'] = np.where((df_test['Likes'] > mean_likes) | (df_test['Replies'] > mean_replies) | (df_test['Retweets'] > mean_retweets), 'positif', 'negatif')

# Memisahkan dataset menjadi data training dan data testing
train_df = df_test.sample(frac=0.9, random_state=1)
test_df = df_test.drop(train_df.index)

# Membuat objek TfidfVectorizer
vectorizer = TfidfVectorizer()

# Melakukan fitting dan transform pada data training
X_train = vectorizer.fit_transform(train_df['Tweet'])
y_train = train_df['Sentimen']

# Melakukan transform pada data testing
X_test = vectorizer.transform(test_df['Tweet'])
y_test = test_df['Sentimen']

# Melatih model Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)

FileNotFoundError: ignored

In [ ]:
# Mengubah teks menjadi vektor fitur numerik menggunakan TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)


NameError: ignored

In [ ]:
# Preprocessing tweet
def preprocess_tweet(tweet):
    # Menghapus karakter non-angka/non-huruf
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    # Mengubah semua huruf menjadi lowercase
    tweet = tweet.lower()
    return tweet

# Menerapkan preprocessing pada tweet
df['Tweet'] = df['Tweet'].apply(preprocess_tweet)

# Melakukan transform pada data testing
X_test = vectorizer.transform(df['Tweet'])

# Melakukan prediksi sentimen
y_pred = nb.predict(X_test)

# Menambahkan kolom sentimen pada dataset
df['Sentimen'] = y_pred

# Menyimpan dataset dengan kolom sentimen baru
df.to_csv("sentimen_hasil.csv", index=False)


NameError: ignored

In [ ]:
# Memprediksi sentimen pada data testing
y_pred = nb.predict(X_test)

In [ ]:
# Menghitung nilai akurasi, presisi, recall, dan f1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='positif')
recall = recall_score(y_test, y_pred, pos_label='positif')
f1 = f1_score(y_test, y_pred, pos_label='positif')

# Mencetak hasil evaluasi
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.78
Precision: 0.75
Recall: 0.125
F1-score: 0.21428571428571427


In [ ]:
df.tail()